# MIKH Model: Stress Relaxation

## Learning Objectives

1. Generate **synthetic stress relaxation** data from calibrated MIKH parameters
2. Understand the **Maxwell backbone** contribution to relaxation behavior
3. Observe the role of **thixotropic restructuring** during relaxation
4. Fit and validate the MIKH model on relaxation data
5. Compare relaxation behavior with pure Maxwell model

## Prerequisites

- NB01: MIKH Flow Curve (provides calibrated parameters)

## Runtime

- Fast demo: ~2-3 minutes
- Full run: ~10-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.ikh import MIKH

# Add examples/utils to path for tutorial utilities
sys.path.insert(0, os.path.join("..", "utils"))
from ikh_tutorial_utils import (
    load_ikh_parameters,
    set_model_parameters,
    generate_synthetic_relaxation,
    save_ikh_results,
    print_convergence_summary,
    print_parameter_comparison,
    compute_fit_quality,
    get_mikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Stress Relaxation in MIKH

Stress relaxation occurs when a constant strain is suddenly applied and maintained. In the MIKH model, the response combines:

### Maxwell Backbone

The Maxwell element provides exponential stress decay:
$$
\sigma(t) = \sigma_0 \exp\left(-\frac{t}{\tau_M}\right), \quad \tau_M = \frac{\eta}{G}
$$

### Thixotropic Effects

During relaxation ($\dot{\gamma} = 0$), the structure evolves according to:
$$
\frac{d\lambda}{dt} = \frac{1 - \lambda}{\tau_{thix}}
$$

Since there's no flow, structure **rebuilds** toward $\lambda = 1$.

### Combined Behavior

The stress relaxation in MIKH is governed by the ODE:
$$
\frac{d\sigma}{dt} = G\dot{\gamma} - \frac{G}{\eta}\sigma - G\dot{\gamma}^p
$$

For relaxation ($\dot{\gamma} = 0$), this simplifies to:
$$
\frac{d\sigma}{dt} = -\frac{G}{\eta}\sigma = -\frac{\sigma}{\tau_M}
$$

However, if the stress is above yield, plastic flow can continue, modifying the relaxation.

## 3. Load Calibrated Parameters

We use parameters calibrated from the flow curve (NB01).

In [ ]:
# Load calibrated parameters from NB01
try:
    calibrated_params = load_ikh_parameters("mikh", "flow_curve")
    print("Loaded calibrated parameters from NB01:")
    for name, val in calibrated_params.items():
        print(f"  {name:15s} = {val:.4g}")
except FileNotFoundError:
    print("NB01 results not found. Using default parameters.")
    calibrated_params = None

In [ ]:
# Create model and set parameters
model = MIKH()
param_names = get_mikh_param_names()

if calibrated_params is not None:
    set_model_parameters(model, calibrated_params)

# Print Maxwell relaxation time
G = model.parameters.get_value("G")
eta = model.parameters.get_value("eta")
tau_M = eta / G
print(f"\nMaxwell relaxation time: tau_M = eta/G = {tau_M:.2g} s")

## 4. Generate Synthetic Data

In [ ]:
# Generate synthetic relaxation data
sigma_0 = 100.0  # Initial stress (Pa)
t_end = 500.0    # End time (s)
n_points = 200
noise_level = 0.02  # 2% noise

t_data, stress_data = generate_synthetic_relaxation(
    model,
    sigma_0=sigma_0,
    t_end=t_end,
    n_points=n_points,
    noise_level=noise_level,
    seed=42,
)

print(f"Generated synthetic data:")
print(f"  Time range: [{t_data.min():.2f}, {t_data.max():.1f}] s")
print(f"  Stress range: [{stress_data.min():.2f}, {stress_data.max():.2f}] Pa")
print(f"  Points: {len(t_data)}")

In [ ]:
# Plot synthetic data
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_data, stress_data, "ko", markersize=5, alpha=0.7, label="Synthetic data")

# Pure Maxwell reference
sigma_maxwell = sigma_0 * np.exp(-t_data / tau_M)
ax.loglog(t_data, sigma_maxwell, "--", lw=2, color="red", alpha=0.7, label=f"Maxwell ($\\tau$ = {tau_M:.2g} s)")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title("Synthetic Stress Relaxation Data", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 5. NLSQ Fitting

In [ ]:
# Create fresh model for fitting
model_fit = MIKH()

t0 = time.time()
model_fit.fit(t_data, stress_data, test_mode="relaxation", sigma_0=sigma_0, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model_fit.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality
stress_pred = model_fit.predict_relaxation(t_data, sigma_0=sigma_0)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

In [ ]:
# Plot fit
t_fine = np.logspace(np.log10(t_data.min()), np.log10(t_data.max()), 200)
stress_fine = model_fit.predict_relaxation(t_fine, sigma_0=sigma_0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(t_data, stress_data, "ko", markersize=5, alpha=0.7, label="Data")
ax.loglog(t_fine, stress_fine, "-", lw=2.5, color="C0", label="MIKH fit")
ax.loglog(t_fine, sigma_0 * np.exp(-t_fine / tau_M), "--", lw=2, color="red", alpha=0.7, label="Maxwell reference")

ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title(f"MIKH Stress Relaxation Fit (R$^2$ = {metrics['R2']:.5f})", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Bayesian Inference with NUTS

In [ ]:
# Bayesian inference
initial_values = {name: model_fit.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model_fit.fit_bayesian(
    t_data,
    stress_data,
    test_mode="relaxation",
    sigma_0=sigma_0,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
all_pass = print_convergence_summary(result, param_names)

In [ ]:
# Trace plots for Maxwell parameters
idata = result.to_inference_data()
maxwell_params = ["G", "eta", "sigma_y0", "delta_sigma_y"]
axes = az.plot_trace(idata, var_names=maxwell_params, figsize=(12, 6))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots (Maxwell/Yield Parameters)", fontsize=14, y=1.00)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# NLSQ vs Bayesian comparison
posterior = result.posterior_samples
print_parameter_comparison(model_fit, posterior, param_names)

## 7. Physical Interpretation

### Relaxation Regimes

1. **Short times** ($t \ll \tau_M$): Stress remains near initial value
2. **Intermediate times** ($t \sim \tau_M$): Exponential decay dominates
3. **Long times** ($t \gg \tau_M$): Stress approaches residual yield stress (if any)

### Thixotropic Effects

During relaxation, the structure rebuilds ($\lambda \to 1$), which can:
- Increase the effective yield stress
- Slow down the late-stage relaxation
- Create a plateau at the structured yield stress

## 8. Save Results

In [ ]:
# Save results
save_ikh_results(model_fit, result, "mikh", "relaxation", param_names)

## Key Takeaways

1. **Stress relaxation** in MIKH is governed by the Maxwell relaxation time $\tau_M = \eta/G$

2. **Thixotropic restructuring** during relaxation can modify the late-stage behavior

3. **Synthetic data** from calibrated parameters allows validation of relaxation predictions

4. **ODE formulation** (Diffrax) provides accurate time integration for relaxation

5. The **Maxwell backbone** is the primary relaxation mechanism, with thixotropy providing corrections

### Next Steps

- **NB04**: Creep response (real ML-IKH data)
- **NB05**: SAOS (synthetic from NB01 calibration)
- **NB06**: LAOS (real PNAS data)